### Import Modules

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Data Preparation

In [ ]:
!unzip FinancialPhraseBank-v1.0.zip

In [ ]:
path = 'FinancialPhraseBank-v1.0/Sentences_75Agree.txt'
df = pd.read_csv(path, delimiter='@',encoding = "ISO-8859-1",index_col=None)
df.columns = ['text','sentiment']

In [ ]:
### Do the label encoding of the sentiments
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df["sentiment"] = le.fit_transform(df['sentiment'])
df.head()

,text,sentiment
0,With the new production plant the company woul...,2
1,"For the last quarter of 2010 , Componenta 's n...",2
2,"In the third quarter of 2010 , net sales incre...",2
3,Operating profit rose to EUR 13.1 mn from EUR ...,2
4,"Operating profit totalled EUR 21.1 mn , up fro...",2


In [ ]:
X = df.text.values.tolist()
y = df.sentiment.values.tolist()

### Sentence Transformers Embeddings

In [ ]:
model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

In [ ]:
sentence_embeddings = model.encode(X)

### Cosine Similarlity for Lines Similarity Check

In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(sentence_embeddings, sentence_embeddings)

In [ ]:
#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

In [ ]:
#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)


In [ ]:
print("Top-5 most similar pairs:")
for score, i, j in all_sentence_combinations[0:5]:
  print("----------PAIR-----------")
  print(X[i])
  print(X[j])
  print(cos_sim[i][j])
  ###print("{} \t {} \t {:.4f}".format(val_text[i], val_text[j], cos_sim[i][j]))

Top-5 most similar pairs:
----------PAIR-----------
Operating profit for the 12-month period decreased from EUR17 .9 m while net sales increased from EUR58 .3 m , as compared to 2007 .
Operating profit for the 12-month period decreased from EUR157 .5 m , while net sales increased from EUR634 .3 m , as compared to 2007 .
tensor(0.9920)
----------PAIR-----------
No financial details of the deployment were disclosed .
No financial or pricing details were disclosed .
tensor(0.9683)
----------PAIR-----------
No financial detail were available .
No financial or pricing details were disclosed .
tensor(0.9431)
----------PAIR-----------
Pretax profit decreased to EUR 33.8 mn from EUR 40.8 mn in the fourth quarter of 2005 .
Vaisala 's net profit for the third quarter of 2007 dropped to 3.0 mln euro ( $ 4.3 mln ) from 6.8 mln euro ( $ 9.8 mln ) for the same period of 2006 .
tensor(0.9412)
----------PAIR-----------
The operating loss amounted to EUR 0.8 mn , compared to a profit of EUR 3.9 mn a ye